In [130]:
from sklearn.model_selection import (cross_val_score, train_test_split, 
                                    RepeatedStratifiedKFold, GridSearchCV)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from prettytable import PrettyTable
import pandas as pd
import numpy as np

#### Đọc file

In [131]:
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Cột Age có 177 giá trị bị trông, Fare không có giá trị bị trống

In [132]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

#### Lặp Logistic Regression với random state của train_test_split thay đổi từ 0 đến 9 
#### Lấy độ chính xác trung bình

In [133]:
def use_logistic_reg(data, has_new_feature=False):
    """
    Hàm áp dụng Logistic Regression cho bài toán Titanic
    data : dataset
    has_new_feature : có tạo thêm đặc trưng mới hay không
    """
    acc = []
    for i in range(0, 10):
        # Nếu có tạo thêm đặc trưng thì sử dung thêm cột Age_NAN
        if has_new_feature:
            X_train, X_test, Y_train, Y_test = train_test_split(
                data[['Age', 'Fare', 'Age_NAN']].fillna(0), data['Survived'],
                test_size=0.3, random_state=i)
        else:
            X_train, X_test, Y_train, Y_test = train_test_split(
                data[['Age', 'Fare']].fillna(0), data['Survived'],
                test_size=0.3, random_state=i)
        # Tạo bộ phân loại và huấn luyện
        classifier = LogisticRegression()
        classifier.fit(X_train, Y_train)
        # Kết quả dự đoán
        Y_pred = classifier.predict(X_test)
        # Thêm vào mảng kết quả
        acc.append(accuracy_score(Y_test, Y_pred))
    # Lấy trung bình
    return np.mean(acc)

Không xử lý gì cả

In [134]:
print(f"{round(use_logistic_reg(df) * 100, 1)} (%)")

65.6 (%)


Dùng median để thay thế dữ liệu trống

In [135]:
def median_way(df):
    data1 = df.copy()
    median = data1['Age'].median()
    data1['Age'] = data1['Age'].fillna(median)
    return data1

Dùng mean để thay thế dữ liệu trống

In [136]:
def mean_way(df):
    data2 = df.copy()
    mean = data2['Age'].mean()
    data2['Age'] = data2['Age'].fillna(mean)
    return data2

Dùng mode để thay thế dữ liệu trống

In [137]:
def mode_way(df):
    data3 = df.copy()
    mode = data3['Age'].mode()[0]
    data3['Age'] = data3['Age'].fillna(mode)
    return data3

Thay thế dữ liệu trống bằng giá trị ngẫu nhiên

In [138]:
def random_way(df):
    data4 = df.copy()
    # Loại bỏ các giá trị trống của cột Age và lấy ra ngẫu nhiên K giá trị
    # K là số lượng phần tử trống đã bị loại bỏ
    random_samples = data4['Age'].dropna().sample(n=data4['Age'].isnull().sum(), random_state=0)
    # Gán chỉ số của K giá trị ngẫu nhiên đó thành chỉ số K phân tử trống cột Age
    random_samples.index = data4[data4['Age'].isnull()].index
    # Thay thế K phần tử trống cột Age thành K giá trị ngẫu nhiên đã chọn
    data4.loc[data4['Age'].isnull(), 'Age'] = random_samples
    return data4

Thay thế dữ liệu trống bằng giá trị đuôi phân bố

In [139]:
def end_of_dist_way(df):
    data5 = df.copy()
    # Giá trị đuôi phân bố = mean + 3 * std
    extreme = data5["Age"].mean() + 3 * data5["Age"].std()
    data5["Age"] = data5["Age"].fillna(extreme)
    return data5

Thay thế dữ liệu trống bằng giá trị bất kì

In [140]:
def arbitrary_way(df, ar_age):
    data6 = df.copy()
    data6["Age"] = data6["Age"].fillna(ar_age)
    return data6

Tạo đặc trưng mới

In [141]:
def new_feature_way(df):
    data7 = df.copy()
    # Tạo cột Age_NAN
    data7['Age_NAN'] = np.where(data7['Age'].isnull(), 1, 0)
    # Thay thế dữ liệu trống cột Age thành giá trị median
    data7["Age"].fillna(data7["Age"].median(), inplace=True)
    return data7

### Xử lý ngoại lệ

In [142]:
# Age có dạng phân bố chuẩn
# Biên dưới : mean - 3 * std   ;   Biên trên : mean + 3 * std
upper_boundary = df['Age'].mean() + 3* df['Age'].std()
lower_boundary = df['Age'].mean() - 3* df['Age'].std()
print(lower_boundary, upper_boundary)

# Fare có dạng phân bố lệch
# Biên dưới : 1st Quantile - 3 * IQR   ;   Biên trên : 3rd Quantile + 3 * IQR
IQR = df["Fare"].quantile(0.75) - df["Fare"].quantile(0.25)
lower_bridge = df['Fare'].quantile(0.25) - (IQR*3)
upper_bridge = df['Fare'].quantile(0.75) + (IQR*3)
print(lower_bridge, upper_bridge)

-13.880374349943303 73.27860964406094
-61.358399999999996 100.2688


Thay thế các giá trị tuổi lớn hơn biên trên bằng 73

Thay thế các giá vé lớn hơn biên trên bằng 100


In [143]:
df_not_outlier = df.copy()
df_not_outlier.loc[df_not_outlier['Age'] >= 73, 'Age'] = 73
df_not_outlier.loc[df_not_outlier['Fare'] >= 100, 'Fare'] = 100

### Áp dụng để thu được 14 loại dữ liệu

In [144]:
# Có Outlier
data_median = median_way(df)
data_mean = mean_way(df)
data_mode = mode_way(df)
data_random = random_way(df)
data_endofdist = end_of_dist_way(df)
data_arbitrary = arbitrary_way(df, ar_age=27)
data_newfeature = new_feature_way(df)
# Không có Outlier
data_median_outlier = median_way(df_not_outlier)
data_mean_outlier = mean_way(df_not_outlier)
data_mode_outlier = mode_way(df_not_outlier)
data_random_outlier = random_way(df_not_outlier)
data_endofdist_outlier = end_of_dist_way(df_not_outlier)
data_arbitrary_outlier = arbitrary_way(df_not_outlier, ar_age=27)
data_newfeature_outlier = new_feature_way(df_not_outlier)

### Thống kê độ chính xác 14 cách

In [145]:
def acc(data, has_new_feature=False):
    # Làm tròn
    return round(use_logistic_reg(data, has_new_feature) * 100, 1)


table = PrettyTable(["TYPE","Median","Mean","Mode","Random",
                    "End of Dist","Arbitrary","New feature"])

table.add_row(["ORIGIN", acc(data_median), acc(data_mean), acc(data_mode), acc(data_random), 
                acc(data_endofdist), acc(data_arbitrary), acc(data_newfeature, True)])

table.add_row(["OUTLIER-PROCESS", acc(data_median_outlier), acc(data_mean_outlier),
                acc(data_mode_outlier), acc(data_random_outlier), acc(data_endofdist_outlier),
                acc(data_arbitrary_outlier), acc(data_newfeature_outlier, True)])

print(">> Đơn vị độ chính xác : %")
print(table)

>> Đơn vị độ chính xác : %
+-----------------+--------+------+------+--------+-------------+-----------+-------------+
|       TYPE      | Median | Mean | Mode | Random | End of Dist | Arbitrary | New feature |
+-----------------+--------+------+------+--------+-------------+-----------+-------------+
|      ORIGIN     |  65.3  | 65.3 | 65.3 |  65.4  |     66.4    |    65.4   |     66.0    |
| OUTLIER-PROCESS |  66.7  | 66.7 | 66.6 |  66.8  |     66.7    |    66.6   |     66.3    |
+-----------------+--------+------+------+--------+-------------+-----------+-------------+


Độ chính xác thấp nhất là 65.3 (%) khi không xử lý ngoại lệ với các phương pháp Median, Mean, Mode, NewFeature.
Do 3 phương pháp Median, Mean, Mode và NewFeature(thay dữ liệu cột trống bằng Median) làm thay đổi phương sai của dữ liệu.

Độ chính xác cao nhất là 66.8 (%) khi xử lý ngoại lệ và áp dụng phương pháp Random.
Trong bài toán này thì phương pháp Random hoạt động tốt do nó làm phương sai của dữ liệu ít bị biến đổi.

### Áp dụng chuẩn hóa (không chuẩn hóa cột Survived)

Chuẩn hóa z-score

In [146]:
from sklearn.preprocessing import StandardScaler

def zscore_norm(data):
    scaler = StandardScaler()
    df_zscore = data[['Age', 'Fare']].copy()
    df_zscore = pd.DataFrame(scaler.fit_transform(df_zscore), columns=df_zscore.columns)
    df_zscore['Survived'] = data['Survived']
    # Nế có cột Age_NAN thì cũng giữ nguyên
    if 'Age_NAN' in data.columns:
        df_zscore['Age_NAN'] = data['Age_NAN']
    return df_zscore

Chuẩn hóa min-max

In [147]:
from sklearn.preprocessing import MinMaxScaler

def minmax_norm(data):
    min_max = MinMaxScaler()
    df_minmax = data[['Age', 'Fare']].copy()
    df_minmax = pd.DataFrame(min_max.fit_transform(df_minmax), columns=df_minmax.columns)
    df_minmax['Survived'] = data['Survived']
    # Nế có cột Age_NAN thì cũng giữ nguyên
    if 'Age_NAN' in data.columns:
        df_minmax['Age_NAN'] = data['Age_NAN']
    return df_minmax

Chuẩn hóa mạnh với ngoại lệ

In [148]:
from sklearn.preprocessing import RobustScaler

def robust_norm(data):
    scaler = RobustScaler()
    df_robust = data[['Age', 'Fare']].copy()
    df_robust = pd.DataFrame(scaler.fit_transform(df_robust), columns=df_robust.columns)
    df_robust['Survived'] = data['Survived']
    # Nế có cột Age_NAN thì cũng giữ nguyên
    if 'Age_NAN' in data.columns:
        df_robust['Age_NAN'] = data['Age_NAN']
    return df_robust

### Thống kê độ chính xác khi áp dụng chuẩn hóa

Áp dụng chuẩn hóa cho các tổ hợp có xử lý outlier

In [149]:
table = PrettyTable(["TYPE", "Median", "Mean", "Mode", "Random",
                    "End of Dist", "Arbitrary", "New feature"])
table.add_row(["z-score", acc(zscore_norm(data_median_outlier)),
                acc(zscore_norm(data_mean_outlier)),
                acc(zscore_norm(data_mode_outlier)),
                acc(zscore_norm(data_random_outlier)),
                acc(zscore_norm(data_endofdist_outlier)),
                acc(zscore_norm(data_arbitrary_outlier)),
                acc(zscore_norm(data_newfeature_outlier), True)])

table.add_row(["min-max", acc(minmax_norm(data_median_outlier)),
               acc(minmax_norm(data_mean_outlier)),
               acc(minmax_norm(data_mode_outlier)),
               acc(minmax_norm(data_random_outlier)),
               acc(minmax_norm(data_endofdist_outlier)),
               acc(minmax_norm(data_arbitrary_outlier)),
               acc(minmax_norm(data_newfeature_outlier), True)])

table.add_row(["robust", acc(robust_norm(data_median_outlier)),
               acc(robust_norm(data_mean_outlier)),
               acc(robust_norm(data_mode_outlier)),
               acc(robust_norm(data_random_outlier)),
               acc(robust_norm(data_endofdist_outlier)),
               acc(robust_norm(data_arbitrary_outlier)),
               acc(robust_norm(data_newfeature_outlier), True)])

print(">> Đơn vị độ chính xác : %")
print(table)

>> Đơn vị độ chính xác : %
+---------+--------+------+------+--------+-------------+-----------+-------------+
|   TYPE  | Median | Mean | Mode | Random | End of Dist | Arbitrary | New feature |
+---------+--------+------+------+--------+-------------+-----------+-------------+
| z-score |  66.6  | 66.6 | 66.6 |  66.8  |     66.7    |    66.6   |     66.3    |
| min-max |  66.9  | 67.0 | 66.9 |  66.9  |     66.5    |    66.8   |     66.2    |
|  robust |  66.6  | 66.7 | 66.6 |  66.8  |     66.7    |    66.6   |     66.3    |
+---------+--------+------+------+--------+-------------+-----------+-------------+


Áp dụng chuẩn hóa Min-Max cho tổ hợp có xử lý outlier và xử lý dữ liệu trống\
thu được độ chính xác cao nhất là 67 (%) ở phương pháp Mean (tăng từ 66,7 (%) lên 67 (%))

# Sử dụng Cross-Validation

In [150]:
def use_logistic_cross_val(data, has_new_feature=False):
    """
    Hàm áp dụng Logistic Regression cho bài toán Titanic
    Chia dữ liệu bằng Cross Validation
    data : dataset
    has_new_feature : có tạo thêm đặc trưng mới hay không
    """
    lr = LogisticRegression()
    # Nếu có tạo thêm đặc trưng thì sử dung thêm cột Age_NAN
    if has_new_feature:
        X = data[['Age', 'Fare', 'Age_NAN']].fillna(0)
    else:
        X = data[['Age', 'Fare']].fillna(0)
    Y = data['Survived']
    # Thực hiện Cross-Validation
    scores = cross_val_score(lr, X, Y, cv=5)
    # Lấy trung bình độ chính xác 5 folds
    return np.mean(scores)

Áp dụng Cross-Validation cho 14 tổ hợp làm sạch

In [151]:
def acc(data, has_new_feature=False):
    # Làm tròn
    return round(use_logistic_cross_val(data, has_new_feature) * 100, 1)


table = PrettyTable(["TYPE", "Median", "Mean", "Mode", "Random",
                    "End of Dist", "Arbitrary", "New feature"])

table.add_row(["ORIGIN", acc(data_median), acc(data_mean), acc(data_mode), acc(data_random),
               acc(data_endofdist), acc(data_arbitrary), acc(data_newfeature, True)])

table.add_row(["OUTLIER-PROCESS", acc(data_median_outlier), acc(data_mean_outlier),
               acc(data_mode_outlier), acc(data_random_outlier), acc(data_endofdist_outlier),
               acc(data_arbitrary_outlier), acc(data_newfeature_outlier, True)])

print(">> Đơn vị độ chính xác : %")
print(table)

>> Đơn vị độ chính xác : %
+-----------------+--------+------+------+--------+-------------+-----------+-------------+
|       TYPE      | Median | Mean | Mode | Random | End of Dist | Arbitrary | New feature |
+-----------------+--------+------+------+--------+-------------+-----------+-------------+
|      ORIGIN     |  66.0  | 65.9 | 66.2 |  66.9  |     66.8    |    66.1   |     66.2    |
| OUTLIER-PROCESS |  67.2  | 67.2 | 67.5 |  67.1  |     67.0    |    67.0   |     66.5    |
+-----------------+--------+------+------+--------+-------------+-----------+-------------+


Độ chính xác cao nhất 67,5 (%) thuộc về trường hợp xử lý trống bằng Mode và có xử lý ngoại lệ

### Áp dụng thêm chuẩn hóa

In [152]:
table = PrettyTable(["TYPE", "Median", "Mean", "Mode", "Random",
                    "End of Dist", "Arbitrary", "New feature"])
table.add_row(["z-score", acc(zscore_norm(data_median_outlier)),
                acc(zscore_norm(data_mean_outlier)),
                acc(zscore_norm(data_mode_outlier)),
                acc(zscore_norm(data_random_outlier)),
                acc(zscore_norm(data_endofdist_outlier)),
                acc(zscore_norm(data_arbitrary_outlier)),
                acc(zscore_norm(data_newfeature_outlier), True)])

table.add_row(["min-max", acc(minmax_norm(data_median_outlier)),
               acc(minmax_norm(data_mean_outlier)),
               acc(minmax_norm(data_mode_outlier)),
               acc(minmax_norm(data_random_outlier)),
               acc(minmax_norm(data_endofdist_outlier)),
               acc(minmax_norm(data_arbitrary_outlier)),
               acc(minmax_norm(data_newfeature_outlier), True)])

table.add_row(["robust", acc(robust_norm(data_median_outlier)),
               acc(robust_norm(data_mean_outlier)),
               acc(robust_norm(data_mode_outlier)),
               acc(robust_norm(data_random_outlier)),
               acc(robust_norm(data_endofdist_outlier)),
               acc(robust_norm(data_arbitrary_outlier)),
               acc(robust_norm(data_newfeature_outlier), True)])

print(">> Đơn vị độ chính xác : %")
print(table)

>> Đơn vị độ chính xác : %
+---------+--------+------+------+--------+-------------+-----------+-------------+
|   TYPE  | Median | Mean | Mode | Random | End of Dist | Arbitrary | New feature |
+---------+--------+------+------+--------+-------------+-----------+-------------+
| z-score |  67.1  | 67.1 | 67.5 |  67.2  |     67.0    |    67.1   |     66.6    |
| min-max |  67.1  | 67.2 | 67.5 |  67.6  |     66.6    |    67.1   |     66.7    |
|  robust |  67.1  | 67.1 | 67.5 |  67.2  |     67.0    |    67.1   |     66.6    |
+---------+--------+------+------+--------+-------------+-----------+-------------+


Khi áp dụng chuẩn hóa thì độ chính xác cao nhất là 67.6 (%) ở phương pháp Random, có xử lý ngoại lệ và chuẩn hóa min-max

# Hyperparameter Tuning

In [153]:
def logistic_hyper_tuning(data, has_new_feature=False):
    # Nếu có tạo thêm đặc trưng thì sử dung thêm cột Age_NAN
    if has_new_feature:
        X = data[['Age', 'Fare', 'Age_NAN']].fillna(0)
    else:
        X = data[['Age', 'Fare']].fillna(0)
    Y = data['Survived']

    lr = LogisticRegression()
    
    # solver = ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
    # penalty = ["l1", "l2", "elasticnet", "none"]

    grid = {
        "solver": ['newton-cg', 'lbfgs', 'liblinear'],
        "penalty": ['l2'],
        "C": [100, 10, 1.0, 0.1, 0.01]
    }
    """
    Tạo Grid Search với các tham số : 
    estimator : thuật toán đánh giá
    param_grid : danh sách các siêu tham số dùng đánh giá mô hình
    n_jobs : dùng -1 để dùng toàn bộ nhân xử lý của Máy tính cho tác vụ này
    cv : Số lượng Fold trong Cross-Validation
    scoring : tiêu chí đánh giá
    error_score : kết quả trả về khi áp dụng 1 phương pháp mà xảy ra lỗi
    """
    grid = GridSearchCV(estimator=lr, param_grid=grid, n_jobs=-1, cv=5, scoring='accuracy', error_score=0)
    result = grid.fit(X, Y)
    
    # Kết quả Tuning tốt nhất
    print(f">> Best: {result.best_score_} using {result.best_params_}")
    # In ra kết quả toàn bộ các phương pháp dùng Tuning
    means = result.cv_results_["mean_test_score"]
    stds = result.cv_results_["std_test_score"]
    params = result.cv_results_["params"]
    table = PrettyTable(["C", "penalty", "solver", "std", "mean"])
    for mean, std, param in zip(means, stds, params):
        table.add_row([param["C"], param["penalty"], param["solver"], std, mean])
    print(table)

In [154]:
logistic_hyper_tuning(minmax_norm(data_random_outlier))

>> Best: 0.6757579561860523 using {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
+------+---------+-----------+----------------------+--------------------+
|  C   | penalty |   solver  |         std          |        mean        |
+------+---------+-----------+----------------------+--------------------+
| 100  |    l2   | newton-cg | 0.057695787956962126 | 0.6712824053731719 |
| 100  |    l2   |   lbfgs   | 0.057695787956962126 | 0.6712824053731719 |
| 100  |    l2   | liblinear | 0.057695787956962126 | 0.6712824053731719 |
|  10  |    l2   | newton-cg | 0.05823068722558555  | 0.6724060008787898 |
|  10  |    l2   |   lbfgs   | 0.05823068722558555  | 0.6724060008787898 |
|  10  |    l2   | liblinear | 0.05823068722558555  | 0.6724060008787898 |
| 1.0  |    l2   | newton-cg | 0.051825010901695064 | 0.6757579561860523 |
| 1.0  |    l2   |   lbfgs   | 0.051825010901695064 | 0.6757579561860523 |
| 1.0  |    l2   | liblinear | 0.05092623337927883  | 0.6746343606804344 |
| 0.1  |    l2 

##### Kết quả cao nhất là 0.6757 ~ 67,6 (%) và xuất hiện ở các trường hợp : 
{'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'} (mặc định)\
{'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}

### Áp dụng các siêu tham số này cho toàn bộ dữ liệu để thu được mô hình tốt nhất

In [155]:
X = data_random_outlier[['Age', 'Fare']]
Y = data_random_outlier['Survived']
# Tạo bộ phân loại và huấn luyện
classifier = LogisticRegression()
classifier.fit(X, Y)
Y_pred = classifier.predict(X)
print(f"{round(accuracy_score(Y, Y_pred)*100, 1)} (%)")

67.6 (%)


# Thêm đặc trưng giới tính để đua Top

Encode giới tính bằng LabelEncoder

In [156]:
from sklearn.preprocessing import LabelEncoder

def sex_encode(data):
    data_top = data.copy()
    le = LabelEncoder()
    data_top["Sex"] = df["Sex"]
    le.fit(data_top["Sex"])
    data_top["Sex_Encode"] = le.transform(data_top["Sex"])
    return data_top

Tạo lại Logistic Regression với Cross-Validation

In [157]:
def use_logistic_data_top(data, has_new_feature=False):
    """
    Hàm áp dụng Logistic Regression cho bài toán Titanic
    Chia dữ liệu bằng Cross Validation
    data : dataset
    has_new_feature : có tạo thêm đặc trưng mới hay không
    """
    lr = LogisticRegression()
    # Nếu có tạo thêm đặc trưng thì sử dung thêm cột Age_NAN
    if has_new_feature:
        X = data[['Age', 'Fare', 'Age_NAN', "Sex_Encode"]].fillna(0)
    else:
        X = data[['Age', 'Fare', "Sex_Encode"]].fillna(0)
    Y = data['Survived']

    # Tạo 5-folds cross-validation lặp 3 lần để tăng hiệu quả
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    # Thực hiện Cross-Validation
    scores = cross_val_score(lr, X, Y, cv=cv)
    # Lấy trung bình độ chính xác 5 folds
    return np.mean(scores)

Đánh giá các tổ hợp đã xử lý outlier

In [158]:
def acc(data, has_new_feature=False):
    # Làm tròn
    return round(use_logistic_data_top(data, has_new_feature) * 100, 1)

table = PrettyTable(["TYPE", "Median", "Mean", "Mode", "Random",
                    "End of Dist", "Arbitrary", "New feature"])
table.add_row(["z-score", acc(sex_encode(zscore_norm(data_median_outlier))),
               acc(sex_encode(zscore_norm(data_mean_outlier))),
               acc(sex_encode(zscore_norm(data_mode_outlier))),
               acc(sex_encode(zscore_norm(data_random_outlier))),
               acc(sex_encode(zscore_norm(data_endofdist_outlier))),
               acc(sex_encode(zscore_norm(data_arbitrary_outlier))),
               acc(sex_encode(zscore_norm(data_newfeature_outlier)), True)])

table.add_row(["min-max", acc(sex_encode(minmax_norm(data_median_outlier))),
               acc(sex_encode(minmax_norm(data_mean_outlier))),
               acc(sex_encode(minmax_norm(data_mode_outlier))),
               acc(sex_encode(minmax_norm(data_random_outlier))),
               acc(sex_encode(minmax_norm(data_endofdist_outlier))),
               acc(sex_encode(minmax_norm(data_arbitrary_outlier))),
               acc(sex_encode(minmax_norm(data_newfeature_outlier)), True)])

table.add_row(["robust", acc(sex_encode(robust_norm(data_median_outlier))),
               acc(sex_encode(robust_norm(data_mean_outlier))),
               acc(sex_encode(robust_norm(data_mode_outlier))),
               acc(sex_encode(robust_norm(data_random_outlier))),
               acc(sex_encode(robust_norm(data_endofdist_outlier))),
               acc(sex_encode(robust_norm(data_arbitrary_outlier))),
               acc(sex_encode(robust_norm(data_newfeature_outlier)), True)])

print(">> Đơn vị độ chính xác : %")
print(table)

>> Đơn vị độ chính xác : %
+---------+--------+------+------+--------+-------------+-----------+-------------+
|   TYPE  | Median | Mean | Mode | Random | End of Dist | Arbitrary | New feature |
+---------+--------+------+------+--------+-------------+-----------+-------------+
| z-score |  78.5  | 78.5 | 78.5 |  78.4  |     78.6    |    78.5   |     78.6    |
| min-max |  78.3  | 78.3 | 78.3 |  78.6  |     78.6    |    78.3   |     78.6    |
|  robust |  78.5  | 78.5 | 78.5 |  78.4  |     78.6    |    78.5   |     78.6    |
+---------+--------+------+------+--------+-------------+-----------+-------------+


Kết quả cao nhất là 78,6 (%) xảy ra cả 3 trường hợp chuẩn hóa của phương pháp EndOfDist và New Feature và ở chuẩn hóa min-max cho phương pháp Random 

Hypermeter tuning cho dữ liệu khi xét thêm giới tính

In [159]:
def logistic_tuning_data_top(data, has_new_feature=False):
    # Nếu có tạo thêm đặc trưng thì sử dung thêm cột Age_NAN
    if has_new_feature:
        X = data[['Age', 'Fare', 'Age_NAN', "Sex_Encode"]].fillna(0)
    else:
        X = data[['Age', 'Fare', "Sex_Encode"]].fillna(0)
    Y = data['Survived']

    lr = LogisticRegression()

    # solver = ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
    # penalty = ["l1", "l2", "elasticnet", "none"]

    grid = {
        "solver": ['newton-cg', 'lbfgs', 'liblinear'],
        "penalty": ['l2'],
        "C": [100, 10, 1.0, 0.1, 0.01]
    }
    
    # Tạo 5-folds cross-validation lặp 3 lần để tăng hiệu quả
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    grid = GridSearchCV(estimator=lr, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy', error_score=0)
    result = grid.fit(X, Y)

    # Kết quả Tuning tốt nhất
    print(f">> Best: {result.best_score_} using {result.best_params_}")
    # In ra kết quả toàn bộ các phương pháp dùng Tuning
    means = result.cv_results_["mean_test_score"]
    stds = result.cv_results_["std_test_score"]
    params = result.cv_results_["params"]
    table = PrettyTable(["C", "penalty", "solver", "std", "mean"])
    for mean, std, param in zip(means, stds, params):
        table.add_row([param["C"], param["penalty"], param["solver"], std, mean])
    print(table)

Tuning cho trường hợp Random-MinMax-Có xử lý outlier

In [160]:
logistic_tuning_data_top(sex_encode(minmax_norm(data_random_outlier)))

>> Best: 0.7867574749440295 using {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
+------+---------+-----------+----------------------+--------------------+
|  C   | penalty |   solver  |         std          |        mean        |
+------+---------+-----------+----------------------+--------------------+
| 100  |    l2   | newton-cg | 0.020689615430597316 | 0.7833887807837967 |
| 100  |    l2   |   lbfgs   | 0.020689615430597316 | 0.7833887807837967 |
| 100  |    l2   | liblinear | 0.020689615430597316 | 0.7833887807837967 |
|  10  |    l2   | newton-cg | 0.020068078685305017 | 0.7841378444542088 |
|  10  |    l2   |   lbfgs   | 0.020068078685305017 | 0.7841378444542088 |
|  10  |    l2   | liblinear | 0.020068078685305017 | 0.7841378444542088 |
| 1.0  |    l2   | newton-cg | 0.01873746348193569  | 0.785629694725169  |
| 1.0  |    l2   |   lbfgs   | 0.01873746348193569  | 0.785629694725169  |
| 1.0  |    l2   | liblinear |  0.0191513304390025  | 0.7845060992195508 |
| 0.1  |    l2 

##### Thu được độ chính xác 0.7867 ~ 78.7 (%) cao hơn trường hợp dùng bộ tham số mặc định 78,6 (%) ở các bộ siêu tham số :
{'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}\
{'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}\
{'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}

# Kết quả tốt nhất

In [161]:
best_data = sex_encode(minmax_norm(data_random_outlier))
lr = LogisticRegression(C=0.1)
# Tạo 5-folds cross-validation lặp 3 lần để tăng hiệu quả
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# Thực hiện Cross-Validation
scores = cross_val_score(lr, best_data[["Age","Fare","Sex_Encode"]], Y, cv=cv)
# Lấy trung bình độ chính xác 5 folds
print(f"{round(np.mean(scores)*100, 1)} (%)")

78.7 (%)
